# Multimodal Gömme ile Sanatsal Stillerin Analizi

*Yazar: [Jacob Marks](https://huggingface.co/jamarks)*
*Çevirmen: [Aylin Gümüş](https://huggingface.co/aylingumus)*

![Art Analysis Cover Image](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_cover_image.jpg)

Görüntüler gibi görsel veriler bilgi açısından inanılmaz derecede zengindir, ancak yapısal olmayan doğası gereği analiz etmesi zordur.

Bu not defterinde, multimodal embedding yöntemlerini ve hesaplanan öznitelikleri kullanarak resimlerdeki sanatsal stillerin nasıl analiz edileceğini keşfedeceğiz. Veri analizi ve görselleştirme için kullanmak üzere FiftyOne'a yükleyeceğimiz 🤗 Hub'dan [WikiArt veri kümesini](https://huggingface.co/datasets/huggan/wikiart) kullanacağız. Verilere birçok farklı açıdan göz atacağız:

- **Görüntü Benzerliği Arama ve Anlamsal (Semantik) Arama**: 🤗 Transformers'tan ön-eğitimli [CLIP](https://huggingface.co/openai/clip-vit-base-patch32) modelini kullanarak multimodal embedding'leri üreteceğiz ve yapısal olmayan aramalara izin vermek için veriyi indeksleyeceğiz.

- **Kümeleme ve Görselleştirme**: Embedding'leri kullanarak görüntüleri sanatsal stillerine göre kümeleyeceğiz ve UMAP boyut indirgeme yöntemini kullanarak sonuçları görselleştireceğiz.

- **Benzersizlik Analizi**: Veri kümesindeki diğer görüntülere ne kadar benzediğine bağlı olarak her görüntü için benzersizlik skoru atamak için embedding'lerimizi kullanacağız.

- **Görüntü Kalitesi Analizi**: Her bir görüntü için parlaklık, kontrast ve doygunluk gibi görüntü kalite metriklerini hesaplayacağız ve bu metriklerin, görüntülerin sanatsal stili ile nasıl ilişkili olduğunu göreceğiz.

## Haydi başlayalım! 🚀

Bu not defterini çalıştırmak için, aşağıdaki kütüphaneleri yüklemelisiniz:

In [ ]:
!pip install -U transformers huggingface_hub fiftyone umap-learn

İndirmeleri ışık hızında yapmak için [HF Transfer](https://pypi.org/project/hf-transfer/) yükleyin:

```bash
pip install hf-transfer
```

Ve `HF_HUB_ENABLE_HF_TRANSFER` ortam değişkenini ayarlayarak etkinleştirin:

```bash
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
```

<div class="alert alert-block alert-info">
<b>Not:</b> Bu not defteri <code>transformers==4.40.0</code>, <code>huggingface_hub==0.22.2</code> ve <code>fiftyone==0.23.8</code> ile test edilmiştir.
</div>

Şimdi bu not defteri için gerekli olan modülleri içe aktaralım:

In [1]:
import fiftyone as fo # base library and app
import fiftyone.zoo as foz # zoo datasets and models
import fiftyone.brain as fob # ML routines
from fiftyone import ViewField as F # for defining custom views
import fiftyone.utils.huggingface as fouh # for loading datasets from Hugging Face

Hub'dan 🤗 FiftyOne'a WikiArt veri kümesini yükleyerek başlayacağız. Bu veri kümesi Hugging Face'in `datasets` kütüphanesi aracılığıyla da yüklenebilir, ancak biz verileri doğrudan Datasets sunucusundan almak için [FiftyOne'ın 🤗 Hub entegrasyonunu](https://docs.voxel51.com/integrations/huggingface.html#huggingface-hub) kullanacağız. Hesaplamaları hızlı yapmak için, sadece ilk $1,000$ örneği indireceğiz.

In [2]:
dataset = fouh.load_from_hub(
    "huggan/wikiart", ## repo_id
    format="parquet", ## for Parquet format
    classification_fields=["artist", "style", "genre"], # columns to store as classification fields
    max_samples=1000, # number of samples to load
    name="wikiart", # name of the dataset in FiftyOne
)

Veri kümesinin ne içerdiğini görmek için özetini ekrana yazdırın:

In [3]:
print(dataset)

Name:        wikiart
Media type:  image
Num samples: 1000
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    artist:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    style:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    genre:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    row_idx:  fiftyone.core.fields.IntField


Veri kümesini [FiftyOne uygulamasında](https://docs.voxel51.com/user_guide/app.html) görselleştirin:

In [ ]:
session = fo.launch_app(dataset)

![WikiArt Dataset](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_wikiart_dataset.jpg)

Haydi, stillerini analiz edeceğimiz artistlerin isimlerini listeleyelim:

In [4]:
artists = dataset.distinct("artist.label")
print(artists)

['Unknown Artist', 'albrecht-durer', 'boris-kustodiev', 'camille-pissarro', 'childe-hassam', 'claude-monet', 'edgar-degas', 'eugene-boudin', 'gustave-dore', 'ilya-repin', 'ivan-aivazovsky', 'ivan-shishkin', 'john-singer-sargent', 'marc-chagall', 'martiros-saryan', 'nicholas-roerich', 'pablo-picasso', 'paul-cezanne', 'pierre-auguste-renoir', 'pyotr-konchalovsky', 'raphael-kirchner', 'rembrandt', 'salvador-dali', 'vincent-van-gogh']


## Benzer Sanat Eserlerini Bulma

Hoşunuza giden bir sanat eseri bulduğunuzda, ona benzer parçalar bulmak istemeniz doğaldır. Bunu vektör embedding'leriyle yapabiliriz! Dahası, multimodal embedding'leri aracılığıyla, bir tablonun ya da bir şiirin bile tanımı olabilecek, verilen bir metin sorgusunu en yakın temsil eden tabloları bulma yeteğini açacağız.

Haydi, görüntüler için Transformers'tan 🤗 ön-eğitimli bir CLIP Vision Transformer (ViT) modelini kullanarak multimodal embedding'leri üretelim. [FiftyOne Brain](https://docs.voxel51.com/user_guide/brain.html)'in `compute_similarity()` fonksiyonunu çalıştırmak, bu embedding'leri hesaplayacak ve bu embedding'leri, veri setinde bir benzerlik indeksi üretmek için kullanacak.

In [5]:
fob.compute_similarity(
    dataset, 
    model="zero-shot-classification-transformer-torch", ## type of model to load from model zoo
    name_or_path="openai/clip-vit-base-patch32", ## repo_id of checkpoint
    embeddings="clip_embeddings", ## name of the field to store embeddings
    brain_key="clip_sim", ## key to store similarity index info
    batch_size=32, ## batch size for inference
    )

Computing embeddings...
 100% |███████████████| 1000/1000 [5.0m elapsed, 0s remaining, 3.3 samples/s]    


<div style="padding: 10px; border-left: 5px solid #0078d4; font-family: Arial, sans-serif; margin: 10px 0;">

Alternatif olarak, modeli doğrudan 🤗 Transformers kütüphanesinden yükleyebilir ve modeli doğrudan kullanabilirsiniz:

```python
from transformers import CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
fob.compute_similarity(
    dataset, 
    model=model,
    embeddings="clip_embeddings", ## name of the field to store embeddings
    brain_key="clip_sim" ## key to store similarity index info
)
```

Bu ve daha fazlası hakkında daha kapsamlı bir rehber için, <a href="https://docs.voxel51.com/integrations/huggingface.html#transformers-library">FiftyOne'ın 🤗 Transformers entegrasyonunu</a> inceleyin.
</div>


FiftyOne uygulamasını yenileyin, örnek listedeki bir görüntünün onay kutusunu seçin ve veri kümesindeki en benzer görüntüleri görmek için fotoğraf ikonuna tıklayın. Arka planda, bu butona tıklamak, seçilen görüntüye en çok benzeyen görüntüleri bulan benzerlik indeksi, önceden hesaplanmış embedding'lere dayanarak bir sorguyu tetikler ve bunları uygulamada gösterir.

![Image Similarity Search](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_image_search.gif)

Bunu, verilen sanat eserine hangi sanat eserlerinin en çok benzediğini görmek için kullanabiliriz. Benzer sanat eserlerini bulmak (kullanıcılara önermek ya da bir koleksiyona eklemek için) ya da yeni bir eser için ilham almak amacıyla faydalı olabilir.

Ama dahası var! CLIP multimodal olduğu için, anlamsal (semantik) aramalar gerçekleştirmek için de kullanabiliriz! Bu, metin sorgularına dayalı görüntü araması yapabileceğimiz anlamına geliyor. Örneğin, "pastel ağaçlar" metniyle arama yapabiliriz ve veri kümesinde bu sorguya benzeyen tüm görüntüleri görebiliriz. Bunu yapmak için FiftyOne uygulamasındaki arama butonuna tıklayın ve bir metin sorgusu girin:

![Semantic Search](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_semantic_search.gif)

Sahne arkasında, metin tokenize edilir, CLIP'in metin kodlayıcısı ile gömülür, ardından veri kümesindeki en benzer görüntüleri bulmak için benzerlik indeksini sorgulamak için kullanılır. Bu, metin sorgularına dayalı görüntü aramak için oldukça etkili bir yöntemdir ve belirli bir tema ya da stille eşleşen görüntüleri bulmak için faydalı olabilir. Ve bu CLIP ile sınırlı değil; görüntüler ve metin için embedding'ler üretebilen 🤗 Transformers'tan herhangi bir CLIP-benzeri model kullanabilirsiniz!

<div class="alert alert-block alert-info">
💡 Etkili vektör araması ve büyük veri kümeleri üzerinde indeksleme için, Fifty One'ın <a href="https://voxel51.com/vector-search">açık kaynak vektör veritabanları ile yerel entegrasyonları</a> var.
</div>


## Kümeleme ve Görselleştirme ile Sanatsal Motiflerin Ortaya Çıkarılması

Benzerlik ve anlamsal (semantik) aramalar yaparak verilerle daha etkili bir şekilde etkileşim kurmaya başlayabiliriz. Ancak bunu bir adım daha ileri götürebilir ve işin içine biraz da denetimsiz öğrenme ekleyebiliriz. Bu, WikiArt veri kümesindeki sanatsal paternleri belirlememize yardımcı olacak; stilistik, topikal ve hatta kelimelerle ifade edilmesi zor olan motifleri bile.

Bunu iki yöntemle yapacağız:

1. **Boyut İndirgeme**: Embedding'lerin boyutunu 2B'ye indirgemek ve verileri bir dağılım grafiğinde görselleştirmek için UMAP yöntemini kullanacağız. Bu, görüntülerin stillerine, türlerine ve artistlerine göre nasıl kümelendiğini görmemizi sağlayacaktır.
2. **Kümeleme**: Görüntüleri embedding'lerine göre kümelemek ve hangi grupların ortaya çıktığını görmek için K-Means kümeleme yöntemini kullanacağız.

Boyut indirgeme için, önceden hesaplanmış embedding'leri parametre olarak geçerek FiftyOne Brain'den `compute_visualization()` fonksiyonunu çalıştıracağız. Boyut indirgeme amacıyla UMAP  yöntemini kullanmak için `method="umap"` olarak belirtiyoruz, ancak PCA veya t-SNE de kullanabiliriz:

In [6]:
fob.compute_visualization(dataset, embeddings="clip_embeddings", method="umap", brain_key="clip_vis")

Generating visualization...


/opt/homebrew/Caskroom/miniforge/base/envs/fdev/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


UMAP( verbose=True)
Tue Apr 30 11:51:45 2024 Construct fuzzy simplicial set
Tue Apr 30 11:51:46 2024 Finding Nearest Neighbors
Tue Apr 30 11:51:47 2024 Finished Nearest Neighbor Search
Tue Apr 30 11:51:48 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Tue Apr 30 11:51:49 2024 Finished embedding


Şimdi FiftyOne uygulamasında veri kümesindeki her görüntü için 2B bir nokta göreceğimiz bir panel açabiliriz. Bu niteliklerin görüntü özelliklerimiz tarafından ne kadar güçlü bir şekilde yakalandığını görmek için, veri kümesindeki noktaları artist veya tür gibi alanlara göre renklendirebiliriz:

![UMAP Visualization](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_visualize_embeddings.gif)

Benzer görüntüleri bir arada gruplayabilmek için embedding'ler üzerinde kümeleme de yapabiliriz — belki bu sanat eserlerinin dominant özellikleri halihazırda var olan etiketler tarafından algılanmamıştır ya da belki tanımlamak istediğimiz farklı alt türler vardır. Verilerimizi kümelemek için [FiftyOne Clustering Plugin](https://github.com/jacobmarks/clustering-plugin)'i indirmemiz gerekecek:

In [ ]:
!fiftyone plugins download https://github.com/jacobmarks/clustering-plugin

Uygulamayı tekrar yenileyerek, kümeleme fonksiyonalitesine uygulamadaki bir operatör yardımıyla erişebiliriz. Operatör listesini açmak için ters tırnak tuşuna basın, "cluster" yazın ve açılan menüden operatörü seçin. Bu, kümeleme algoritmasını, hiperparametreleri ve kümelenecek alanı belirleyebileceğimiz etkileşimli bir panel açacaktır. Basit tutmak için, $10$ küme ile K-Means kümelemesini kullanacağız:

Böylelikle kümeleri uygulamada görselleştirebilir ve görüntülerin embedding'lerine göre nasıl gruplandığını görebiliriz:

![K-means Clustering](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_clustering.gif)

Bazı kümelerin artisti, bazılarının ise türü veya stili temel aldığını görebiliriz. Diğerleri ise daha soyut ve veriden ilk bakışta anlaşılamayan alt türleri veya diğer gruplamaları temsil edebilir.

## Identifying the Most Unique Works of Art

One interesting question we can ask about our dataset is how *unique* each image is. This question is important for many applications, such as recommending similar images, detecting duplicates, or identifying outliers. In the context of art, how unique a painting is could be an important factor in determining its value.

While there are a million ways to characterize uniqueness, our image embeddings allow us to quantitatively assign each sample a uniqueness score based on how similar it is to other samples in the dataset. Explicitly, the FiftyOne Brain's `compute_uniqueness()` function looks at the distance between each sample's embedding and its nearest neighbors, and computes a score between $0$ and $1$ based on this distance. A score of $0$ means the sample is nondescript or very similar to others, while a score of $1$ means the sample is very unique.

In [7]:
fob.compute_uniqueness(dataset, embeddings="clip_embeddings") # compute uniqueness using CLIP embeddings

Computing uniqueness...
Uniqueness computation complete


We can then color by this in the embeddings panel, filter by uniqueness score, or even sort by it to see the most unique images in the dataset:

In [ ]:
most_unique_view = dataset.sort_by("uniqueness", reverse=True)
session.view = most_unique_view.view() # Most unique images

![Most Unique Images](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_most_unique.jpg)

In [ ]:
least_unique_view = dataset.sort_by("uniqueness", reverse=False)
session.view = least_unique_view.view() # Least unique images

![Least Unique Images](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_least_unique.jpg)

Going a step further, we can also answer the question of which artist tends to produce the most unique works. We can compute the average uniqueness score for each artist across all of their works of art:

In [8]:
artist_unique_scores = {
    artist: dataset.match(F("artist.label") == artist).mean("uniqueness")
    for artist in artists
}

sorted_artists = sorted(
    artist_unique_scores, key=artist_unique_scores.get, reverse=True
)

for artist in sorted_artists:
    print(f"{artist}: {artist_unique_scores[artist]}")

Unknown Artist: 0.7932221632002723
boris-kustodiev: 0.7480731948424676
salvador-dali: 0.7368807620414014
raphael-kirchner: 0.7315448102204755
ilya-repin: 0.7204744626806383
marc-chagall: 0.7169373812321908
rembrandt: 0.715205220292227
martiros-saryan: 0.708560775790436
childe-hassam: 0.7018343391132756
edgar-degas: 0.699912746806587
albrecht-durer: 0.6969358680800216
john-singer-sargent: 0.6839955708720844
pablo-picasso: 0.6835137858302969
pyotr-konchalovsky: 0.6780653000855895
nicholas-roerich: 0.6676504687452387
ivan-aivazovsky: 0.6484361530090199
vincent-van-gogh: 0.6472004520699081
gustave-dore: 0.6307283287457358
pierre-auguste-renoir: 0.6271467146993583
paul-cezanne: 0.6251076007168186
eugene-boudin: 0.6103397516167454
camille-pissarro: 0.6046182609119615
claude-monet: 0.5998234558947573
ivan-shishkin: 0.589796389836674


It would seem that the artist with the most unique works in our dataset is Boris Kustodiev! Let's take a look at some of his works:

In [10]:
kustodiev_view = dataset.match(F("artist.label") == "boris-kustodiev")
session.view = kustodiev_view.view()

![Boris Kustodiev Artwork](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_kustodiev_view.jpg)

## Sanatın Görsel Özelliklerle Karakterize Edilmesi

To round things out, let's go back to the basics and analyze some core qualities of the images in our dataset. We'll compute standard metrics like brightness, contrast, and saturation for each image and see how these metrics correlate with the artistic style and genre of the art pieces.

To run these analyses, we will need to download the [FiftyOne Image Quality Plugin](https://github.com/jacobmarks/image-quality-issues):

In [ ]:
!fiftyone plugins download https://github.com/jacobmarks/image-quality-issues/

Refresh the app and open the operators list again. This time type `compute` and select one of the image quality operators. We'll start with brightness:

![Compute Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_compute_brightness.gif)

When the operator finishes running, we will have a new field in our dataset that contains the brightness score for each image. We can then visualize this data in the app:

![Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_brightness.gif)

We can also color by brightness, and even see how it correlates with other fields in the dataset like style:

![Style by Brightness](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_style_by_brightness.gif)

Now do the same for contrast and saturation. Here are the results for saturation:

![Filter by Saturation](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/art_analysis_filter_by_saturation.jpg)

Hopefully this illustrates how not everything boils down to applying deep neural networks to your data. Sometimes, simple metrics can be just as informative and can provide a different perspective on your data 🤓!

<div class="alert alert-block alert-info">
📚 For larger datasets, you may want to <a href="https://docs.voxel51.com/plugins/using_plugins.html#delegated-operations">delegate the operations</a> for later execution.
</div>

## What's Next?

In this notebook, we've explored how to use multimodal embeddings, unsupervised learning, and traditional image processing techniques to analyze artistic styles in images. We've seen how to perform image similarity and semantic searches, cluster images based on their style, analyze the uniqueness of images, and compute image quality metrics. These techniques can be applied to a wide range of visual datasets, from art collections to medical images to satellite imagery. Try [loading a different dataset from the Hugging Face Hub](https://docs.voxel51.com/integrations/huggingface.html#loading-datasets-from-the-hub) and see what insights you can uncover!

If you want to go even further, here are some additional analyses you could try:

- **Zero-Shot Classification**: Use a pre-trained vision-language model from 🤗 Transformers to categorize images in the dataset by topic or subject, without any training data. Check out this [Zero-Shot Classification tutorial](https://docs.voxel51.com/tutorials/zero_shot_classification.html) for more info.
- **Image Captioning**: Use a pre-trained vision-language model from 🤗 Transformers to generate captions for the images in the dataset. Then use this for topic modeling or cluster artwork based on embeddings for these captions. Check out FiftyOne's [Image Captioning Plugin](https://github.com/jacobmarks/fiftyone-image-captioning-plugin) for more info.

### 📚 Kaynaklar

- [FiftyOne 🤝 🤗 Hub Integration](https://docs.voxel51.com/integrations/huggingface.html#huggingface-hub)
- [FiftyOne 🤝 🤗 Transformers Integration](https://docs.voxel51.com/integrations/huggingface.html#transformers-library)
- [FiftyOne Vector Search Integrations](https://voxel51.com/vector-search/)
- [Visualizing Data with Dimensionality Reduction Techniques](https://docs.voxel51.com/tutorials/dimension_reduction.html)
- [Clustering Images with Embeddings](https://docs.voxel51.com/tutorials/clustering.html)
- [Exploring Image Uniqueness with FiftyOne](https://docs.voxel51.com/tutorials/uniqueness.html)

## FiftyOne Açık Kaynak Projesi

[FiftyOne](https://github.com/voxel51/fiftyone/) is the leading open source toolkit for building high-quality datasets and computer vision models. With over 2M downloads, FiftyOne is trusted by developers and researchers across the globe.

💪 The FiftyOne team welcomes contributions from the open source community! If you're interested in contributing to FiftyOne, check out the [contributing guide](https://github.com/voxel51/fiftyone/blob/develop/CONTRIBUTING.md).